# Snake Multiplayer

Snake Multipayer zwsichen zwei ESP-32-Geräten via Bluetooth Komunikation.

## Anforderungen
* Snake Game
* Stabile Kommunikation via Bluetooth
* Score Anzeige

## Pin-Layout

<img src="./assets/esp.png" width="500">

* Pull-Up-Schaltung

In [32]:
%serialconnect %serialconnect --port=/dev/cu.usbserial-0001 --baud=115200 

Connecting to --port=/dev/cu.usbserial-0001 --baud=115200 
Ready.


## Einbinden der benötigten Bibliotheken

In [33]:
import machine
import random
import uasyncio as asyncio
import struct
from time import sleep
import json

## Playground

Zeichne Punkt und Score auf Display

In [34]:
class Coordinate:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [35]:
class Playground:
    DISPLAY_WIDTH = 128
    DISPLAY_HEIGHT = 64

    BLOCK_SIZE = 4

    GAME_WIDTH = int(DISPLAY_WIDTH / BLOCK_SIZE)
    GAME_HEIGHT = int(DISPLAY_HEIGHT / BLOCK_SIZE)

    def __init__(self, display):
        self.display = display
        pass

    def drawScore(self, score):
        self.display.text(f"Score {str(score)}", 0, 0)

    def drawCoordinate(self, coordintate: Coordinate):
        self.display.fill_rect(coordintate.x * self.BLOCK_SIZE, coordintate.y * self.BLOCK_SIZE, self.BLOCK_SIZE , self.BLOCK_SIZE)

In [36]:
import display

# Erstellen eines Display Objekts
display = display.Display()

# Erstellen eines Playground Objekts mit übergabe des erstellten Displays
playground = Playground(display)

display.clear()

# Neue Koordinate anlegen - übergabe von x und y Koordinaten
coordinate = Coordinate(20, 10)

# Koordinate und Test-Score auf Display zeichnen
playground.drawCoordinate(coordinate)
playground.drawScore(5)

display.show()

<img src="./assets/dot.png" width="500">

## Zeichne einen Apfel


### Was ist eine Klasse
Eine Klasse definiert Eigenschaften und Methoden. Mit einer Klasse können Objekte mit den gegebenen Eigenschaften angelegt werden.  
Eine Klasse kann als Schablone oder Bauplan für eine Gruppe von Objekten dienen.

In [43]:
class Apple:    
    def __init__(self, playground):
        self.playground = playground
        x = random.randint(0, self.playground.GAME_WIDTH - 1)
        y = random.randint(0, self.playground.GAME_HEIGHT - 1)
        self.coordinate = Coordinate(x, y)
    
    def drawApple(self):
        self.playground.drawCoordinate(self.coordinate)

In [66]:
import display
display = display.Display()
playground = Playground(display)

apple = Apple(playground)
apple.drawApple()

display.show()

## Snake

In [38]:
class Snake:
    START_SNAKE_LENGTH = 2

    def __init__(self, playground: Playground, startPos: Coordinate):
        self.tails = []
        self.length = self.START_SNAKE_LENGTH
        self.direction = 2 # 0 = up, 1 down, 2 left, 3 right
        self.score = 0
        self.tails.append(startPos)
        self.playground = playground


    def getScore(self) -> int:
        return self.score

    def changeDirection(self, newDirection):
        if (self.direction == 0 and newDirection == 1):
            return
        elif (self.direction == 1 and newDirection == 0):
            return
        elif (self.direction == 2 and newDirection == 3):
            return
        elif (self.direction == 3 and newDirection == 2):
            return

        self.direction = newDirection

    def mooveSnake(self):
        head = self.tails[-1]
        newPos = Coordinate(head.x, head.y)

        if (self.direction == 0):
            newPos.y = head.y - 1
            if newPos.y < 0:
                newPos.y = self.playground.GAME_HEIGHT - 1
        elif (self.direction == 1):
            newPos.y = head.y + 1
            if newPos.y >= self.playground.GAME_HEIGHT:
                newPos.y = 0
        elif (self.direction == 2):
            newPos.x = head.x - 1
            if newPos.x < 0:
                newPos.x = self.playground.GAME_WIDTH - 1
        elif (self.direction == 3):
            newPos.x = head.x + 1
            if newPos.x >= self.playground.GAME_WIDTH:
                newPos.x = 0

        if (len(self.tails) >= self.length):
            self.tails.pop(0)

        self.tails.append(newPos)

    def detectAppleCollision(self, apple: Apple) -> bool:
        collisionDetected = self.detectCollission(apple.coordinate)
        if collisionDetected:
            self.length += 2
            self.score += 1
    
        return collisionDetected
    
    def detectGameOver(self, otherSnakeTails):
        head = self.tails[-1]
        
        gameOver = False
        if len(self.tails) > self.START_SNAKE_LENGTH:
            for pos in self.tails[:-2]:
                if pos.x == head.x and pos.y == head.y:
                    gameOver = True

            for pos in otherSnakeTails:
                if pos.x == head.x and pos.y == head.y:
                    gameOver = True
                    

        if gameOver == True:
            for i in range(0, max(self.START_SNAKE_LENGTH, len(self.tails) - self.START_SNAKE_LENGTH)):
                self.tails.pop(0)
            self.length = self.START_SNAKE_LENGTH
            self.score = 0


    def detectCollission(self, coordinate) -> bool:
        head = self.tails[-1]
        return head.x == coordinate.x and head.y == coordinate.y

    def drawSnake(self):
        for tail in self.tails:
            self.playground.drawCoordinate(tail)

## Fertiges Snake Game (Single Player)

In [70]:
import display
display = display.Display()
playground = Playground(display)

snake = Snake(playground, Coordinate(10, 10))
apple = Apple(playground)

downBtn = Pin(14, Pin.IN, Pin.PULL_UP)
rightBtn = Pin(21, Pin.IN, Pin.PULL_UP)
leftBtn = Pin(18, Pin.IN, Pin.PULL_UP)
upBtn = Pin(25, Pin.IN, Pin.PULL_UP)

while True:
    display.clear()

    if downBtn.value() == 0:
        snake.changeDirection(1)
    if rightBtn.value() == 0:
        snake.changeDirection(3)
    if leftBtn.value() == 0:
        snake.changeDirection(2)
    if upBtn.value() == 0:
        snake.changeDirection(0)

    apple.drawApple()

    snake.mooveSnake()
    snake.drawSnake()

    if snake.detectAppleCollision(apple):
        apple = Apple(playground)

    snake.detectGameOver([])

    playground.drawScore(snake.getScore())    

    display.show()

    sleep(0.1)


.....................
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: [Errno 6] Device not configured


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: [Errno 6] Device not configured



# Multiplayer mit Bluetooth-Verbindung

## GATT-Server (Generic Attribute Profile)

<img src="./assets/bluetooth.png" width="50">

<img src="./assets/bleservice.png" width="300">

In [39]:
import aioble
from bluetooth import UUID

class BleService:

    ADVERTISING_INTERVAL_MILLISECONDS = 250_000
    GATT_APPEARANCE = 768

    NAME = "SnakeMultiplayer"

    SNAKE_SERVICE_UUID = UUID("ca07faee-7e95-4856-b44b-bbaef52ec7b4")

    APPLE_POSITION_CHARACTERISTIC_UUID = UUID("cec23f9f-cdc0-4577-8798-8dd4b01724d8")
    SNAKE1_POSITIONS_CHARACTERISTIC_UUID = UUID("e72c988f-7279-4b82-b808-42884a4ba48f")
    SNAKE2_POSITIONS_CHARACTERISTIC_UUID = UUID("1673be03-ef60-4fb9-869c-4513a8e212f1")
    SNAKE2_DIRECTIONS_CHARACTERISTIC_UUID = UUID("1533130f-7251-4a87-a976-1b5bc0fae798")

    snake1TransmittedLength = 0
    snake2TransmittedLength = 0

    def __init__(self):
        self.connection = None
        self.device_connected = False

        self.gattSnakeService = aioble.Service(self.SNAKE_SERVICE_UUID)

        self.snake1PositionsCharacteristic = aioble.Characteristic(
            self.gattSnakeService,
            self.SNAKE1_POSITIONS_CHARACTERISTIC_UUID,
            read=True,
            notify=True)
        
        self.snake2PositionsCharacteristic = aioble.Characteristic(
            self.gattSnakeService,
            self.SNAKE2_POSITIONS_CHARACTERISTIC_UUID,
            read=True,
            notify=True)
        
        self.snake2DirectionsCharacteristic = aioble.Characteristic(
            self.gattSnakeService,
            self.SNAKE2_DIRECTIONS_CHARACTERISTIC_UUID,
            read=False,
            write=True, 
            notify=True, capture=True)
        
        self.applePositionCharacteristic = aioble.Characteristic(
            self.gattSnakeService,
            self.APPLE_POSITION_CHARACTERISTIC_UUID,
            read=True,
            notify=True)
        
        aioble.register_services(self.gattSnakeService)

    async def getSnake2Direction(self):
        value =  self.snake2DirectionsCharacteristic.read()

        if (value):
            val = int.from_bytes(value, "little")
            return int(val)
        else:
            return 1 # default value

    def updateSnake1Tails(self, length: int, tails: list):
        self._udpateSnakeTails(length, self.snake1TransmittedLength, tails, self.snake1PositionsCharacteristic)
        self.snake1TransmittedLength = len(tails)
    
    def updateSnake2Tails(self, length: int, tails: list):
        self._udpateSnakeTails(length, self.snake2TransmittedLength, tails, self.snake2PositionsCharacteristic)
        self.snake2TransmittedLength = len(tails)
    
    def updateApplePosition(self, coordinate: Coordinate):
        self.applePositionCharacteristic.write(f"[{coordinate.x}, {coordinate.y}]")
        self.applePositionCharacteristic.notify(self.connection)

    def _udpateSnakeTails(self, length: int, transmitted_length: int, tails: list, characteristic):
        tailsToTransmitt = tails

        if (transmitted_length > 0):
            tailsToTransmitt = [tails[-1]]

        parsedTails = list(map(lambda d: (d.x, d.y), tailsToTransmitt))
        sendData = [length, parsedTails]
        sendData = str(json.dumps([length, parsedTails])).replace(" ", "")
        characteristic.write(sendData)
        characteristic.notify(self.connection)

    async def advertise(self):
        if self.connection is not None:
            self.device_connected = self.connection.is_connected()

        if self.device_connected is False:
            self.connection = await aioble.advertise(
                self.ADVERTISING_INTERVAL_MILLISECONDS,
                name=self.NAME,
                services=[self.SNAKE_SERVICE_UUID],
                appearance=self.GATT_APPEARANCE,
            )
            self.device_connected = True



*** Sending Ctrl-C



# Fertiger Snake Multiplayer (Server)

In [ ]:
playground = Playground(display)

snakes = [
    Snake(playground, Coordinate(10, 10)),
    Snake(playground, Coordinate(20, 10))
]

apple = Apple(playground)

downBtn = Pin(14, Pin.IN, Pin.PULL_UP)
rightBtn = Pin(21, Pin.IN, Pin.PULL_UP)
leftBtn = Pin(18, Pin.IN, Pin.PULL_UP)
upBtn = Pin(25, Pin.IN, Pin.PULL_UP)

bleService = BleService()

async def main():
    global bleService, display, downBtn, rightBtn, leftBtn, upBtn, apple, snakes, playground, draw_score

    mySnake = snakes[0]
    otherSnake = snakes[1]

    while True:
        display.clear()
        
        if bleService.device_connected is False:
            display.text("Waiting for", 0, 0)
            display.text("connection", 0, 20)
            display.show()

        await bleService.advertise()

        if downBtn.value() == 0:
            mySnake.changeDirection(1)
        if rightBtn.value() == 0:
            mySnake.changeDirection(3)
        if leftBtn.value() == 0:
            mySnake.changeDirection(2)
        if upBtn.value() == 0:
            mySnake.changeDirection(0)

        otherSnake.changeDirection(await bleService.getSnake2Direction())

        apple.drawApple()

        for snake in snakes:
            snake.mooveSnake()
            snake.drawSnake()
            if snake.detectAppleCollision(apple):
                apple = Apple(playground)

            if snake == mySnake:
                snake.detectGameOver(otherSnake.tails)
            else:
                snake.detectGameOver(mySnake.tails)

            if (snake == mySnake):
                playground.drawScore(snake.getScore())

        bleService.updateSnake1Tails(mySnake.length, mySnake.tails)
        bleService.updateSnake2Tails(otherSnake.length, otherSnake.tails)
        bleService.updateApplePosition(apple.coordinate)

        display.show()
        time.sleep(0.1)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

...................................................................................